# Custom Estimator with Keras

**WARNING: This notebook currently fails**

- It's possible to use `tf.feature_column` with `tf.keras` as demonstrated [here](https://colab.sandbox.google.com/drive/1knOAOGwPQTguUj8kh8xSHu05AGlPQYk7)
- It's also possible to use `tf.keras` with `tf.estimator` as demonstrated [here](https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive/09_sequence/txtclsmodel/trainer/model.py)
- This notebook attemps to use `tf.feature_column`, `tf.keras` and `tf.estimator` all together, but **currently the code fails.** 
- It's possible that this is something we'll need to wait on TF 2.0 to fix, but I invite you to try to get it working with TF 1.12

**Learning Objectives**
- Learn how to create custom estimator using tf.keras
    
Up until now we've been limited in our model architectures to premade estimators. But what if we want more control over the model? 

We can use the popular Keras API to create a custom model and then convert it to an estimator using `tf.keras.estimator.model_to_estimator()`. 

This gives us access to all the flexibility of Keras for creating deep learning models, but also the production readiness of the estimator framework!

In [1]:
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)

1.12.0


## 2) Train and Evaluate Input Functions

Same as before

In [2]:
CSV_COLUMN_NAMES = ['fare_amount','dayofweek','hourofday','pickuplon','pickuplat','dropofflon','dropofflat','passengers']
CSV_DEFAULTS = [[0.0],[1],[0],[-74.0], [40.0], [-74.0], [40.7], [1]]

def read_dataset(csv_path):
    def _parse_row(row):
        # Decode the CSV row into list of TF tensors
        fields = tf.decode_csv(row, record_defaults=CSV_DEFAULTS)

        # Pack the result into a dictionary
        features = dict(zip(CSV_COLUMN_NAMES, fields))
        
        # NEW: Add engineered features
        features = add_engineered_features(features)
        
        # Separate the label from the features
        label = features.pop('fare_amount') # remove label from features and store

        return features, label
    
    # Create a dataset containing the text lines.
    dataset = tf.data.Dataset.list_files(csv_path) # (i.e. data_file_*.csv)
    dataset = dataset.flat_map(lambda filename:tf.data.TextLineDataset(filename).skip(1))

    # Parse each CSV row into correct (features,label) format for Estimator API
    dataset = dataset.map(_parse_row)
    
    return dataset

def train_input_fn(csv_path, batch_size=128):
    #1. Convert CSV into tf.data.Dataset  with (features,label) format
    dataset = read_dataset(csv_path)
      
    #2. Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
   
    return dataset

def eval_input_fn(csv_path, batch_size=128):
    #1. Convert CSV into tf.data.Dataset  with (features,label) format
    dataset = read_dataset(csv_path)

    #2.Batch the examples.
    dataset = dataset.batch(batch_size)
   
    return dataset

## 3) Feature Engineering

Same as before except we use `feature_column_v2` which has Keras support

In [2]:
from tensorflow.python.feature_column import feature_column_v2 as fc # NEW

# One hot encode dayofweek and hourofday
fc_dayofweek = fc.categorical_column_with_identity('dayofweek', num_buckets = 8)
fc_hourofday = fc.categorical_column_with_identity('hourofday', num_buckets = 24)

# Cross features to get combination of day and hour
fc_day_hr = fc.crossed_column([fc_dayofweek, fc_hourofday], 24 * 7)

# Bucketize latitudes and longitudes
NBUCKETS = 16
latbuckets = np.linspace(38.0, 42.0, NBUCKETS).tolist()
lonbuckets = np.linspace(-76.0, -72.0, NBUCKETS).tolist()
fc_bucketized_plat = fc.bucketized_column(fc.numeric_column('pickuplon'), lonbuckets)
fc_bucketized_plon = fc.bucketized_column(fc.numeric_column('pickuplat'), latbuckets)
fc_bucketized_dlat = fc.bucketized_column(fc.numeric_column('dropofflon'), lonbuckets)
fc_bucketized_dlon = fc.bucketized_column(fc.numeric_column('dropofflat'), latbuckets)

def add_engineered_features(features):
    features['latdiff'] = features['pickuplat'] - features['dropofflat'] # East/West
    features['londiff'] = features['pickuplon'] - features['dropofflon'] # North/South
    features['euclidean_dist'] = tf.sqrt(features['latdiff']**2 + features['londiff']**2)

    return features

feature_cols = [
  #1. Engineered using tf.feature_column module
  fc.indicator_column(fc_day_hr),
  fc_bucketized_plat,
  fc_bucketized_plon,
  fc_bucketized_dlat,
  fc_bucketized_dlon,
  #2. Engineered in input functions
  fc.numeric_column('latdiff'),
  fc.numeric_column('londiff'),
  fc.numeric_column('euclidean_dist') 
]

## 4) Build Custom Keras Model

Build a Keras model as described [here](https://www.tensorflow.org/guide/keras). The only special consideration is because we're using `tf.feature_column` our first layer must be `FeatureLayer` and takes the list of feature columns as input. 

In [3]:
model = tf.keras.Sequential()
model.add(fc.FeatureLayer(feature_cols))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation=None))

def rmse(y_true, y_pred): # Root Mean Squared Error
  return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='mean_squared_error',
              metrics=[rmse])

## 7) Train and Evaluate

Note the use of `tf.keras.estimator.model_to_estimator` to create our estimator. It takes as arguments the compiled keras model, the OUTDIR, and optionally a `tf.estimator.Runconfig`

In [5]:
from google.datalab.ml import TensorBoard
TensorBoard().start('taxi_trained')

TensorBoard was started successfully with pid 12096. Click here to access it.

12096

In [ ]:
%%time
OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model,
    model_dir = OUTDIR,
    config = tf.estimator.RunConfig(
          tf_random_seed=1, # for reproducibility
          save_checkpoints_steps=100 # checkpoint every N steps
    )
)
    
train_spec=tf.estimator.TrainSpec(
                   input_fn = lambda:train_input_fn('./taxi-train.csv'),
                   max_steps = 500)


eval_spec=tf.estimator.EvalSpec(
                   input_fn=lambda:eval_input_fn('./taxi-valid.csv'),
                   steps = None,
                   start_delay_secs=1, # wait at least N seconds before first evaluation (default 120)
                   throttle_secs=1, # wait at least N seconds before each subsequent evaluation (default 600)
                   exporters = None) # export SavedModel once at the end of training

tf.logging.set_verbosity(tf.logging.INFO) # so loss is printed during training
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

## Cleanup

In [7]:
if len(TensorBoard.list())>0:
  [TensorBoard().stop(pid)for pid in TensorBoard.list()['pid']]
else: print('No TensorBoard instances to stop')

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License